In [67]:
git.status()

NameError: name 'git' is not defined

In [68]:
from datetime import datetime
import gc
import json
import os
from pathlib import Path
import re
import subprocess
import sys
import time

import ipywidgets as widgets
from google.cloud import storage, bigquery
from google.cloud.bigquery import SchemaField
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm

AUTO = tf.data.experimental.AUTOTUNE
BUCKET = 'riiid-caleb'
DATASET = 'data'
LOCATION = 'US'
KAGGLE_SUBMIT_DATASET = 'riiid-submission-private'
PROJECT = 'riiid-caleb'
REPO = 'riiid_2020'
NOT_KAGGLE = os.getenv('KAGGLE_URL_BASE') is None

# if NOT_KAGGLE:
#     from google.colab import drive
#     DRIVE = Path('/content/drive/My Drive')
#     if not DRIVE.exists():
#         drive.mount(str(DRIVE.parent))
#     sys.path.append(str(DRIVE))
#     g_creds_path = 'credentials/riiid-caleb-faddd0c9d900.json'
#     os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(DRIVE/g_creds_path)

bucket = storage.Client(project=PROJECT).get_bucket(BUCKET)
dataset = bigquery.Dataset(f'{PROJECT}.{DATASET}')
bq_client = bigquery.Client(project=PROJECT, location=LOCATION)

if NOT_KAGGLE:
    CONFIG = json.loads(bucket.get_blob('config.json').download_as_string())
    os.environ = {**os.environ, **CONFIG}
    sys.path.append('/home/jupyter')
    from riiid_2020.utilities import check_packages, Git
    from riiid_2020.bqhelpers import BQHelper
    from riiid_2020.queries import Queries
    
    git = Git(REPO, CONFIG.get('GIT_USERNAME'), CONFIG.get('GIT_PASSWORD'),
          CONFIG.get('EMAIL'), '')

#     packages = {
#         'comet-ml': ('3.2.5',''),
#         'gcsfs': ('0.7.1',''),
#         'kaggle': ('1.5.9',''),
#         'plotly': ('4.12.0',''),
# #         'xgboost': ('1.2.0','')
#         # 'lightgbm': ('3.1.0',' --install-option=--gpu')
#     }
#     check_packages(packages)

#     from comet_ml import APIExperiment, Experiment
#     from kaggle.api.kaggle_api_extended import KaggleApi
#     kaggle_api = KaggleApi()
#     kaggle_api.authenticate()

# import lightgbm as lgb
import plotly
import plotly.express as px
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MultiLabelBinarizer
import xgboost as xgb
pd.options.plotting.backend = 'plotly'

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
# from cloud_tpu_client import Client

# c = Client()
# c.configure_tpu_version(tf.__version__, restart_type='ifNeeded')

def get_strategy():

    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver('tpu-1')
        print('Running on TPU ', tpu.master())
    except:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
    
    else:
        strategy = tf.distribute.get_strategy()
        for d in tf.config.list_physical_devices():
            print(d)
            
    return strategy

strategy = get_strategy()

In [ ]:
folds = range(5)

if False:
    df_data = pd.read_gbq((f"""
        WITH data AS (
        SELECT row_id, user_id, content_id, answered_correctly,
        ROW_NUMBER() OVER(PARTITION BY user_id, content_id ORDER BY timestamp DESC) row_num
        FROM riiid-caleb.data.train t
        JOIN riiid-caleb.data.folds f
        ON t.user_id = f.user_id_s
        WHERE content_type_id = 0
        AND t.task_container_id < f.task_container_id_min
        AND f.fold in ({(', '.join(list(map(str, folds))))}) 
        )
        SELECT row_id, user_id, content_id, answered_correctly
        FROM data
        WHERE row_num = 1
        """, '_q_'), use_bqstorage_api=True, progress_bar_type='tqdm_notebook')
    df_data.to_pickle('df_data.pkl')
else:
    df_data = pd.read_pickle('df_data.pkl')

In [ ]:
if False:
    df_data = pd.read_gbq((f"""
        WITH data AS (
        SELECT row_id, user_id, content_id, answered_correctly,
        ROW_NUMBER() OVER(PARTITION BY user_id, content_id ORDER BY timestamp DESC) row_num
        FROM data.train t
        JOIN data.folds f
        ON t.user_id = f.user_id_s
        WHERE content_type_id = 0
        )
        SELECT row_id, user_id, content_id, answered_correctly
        FROM data
        WHERE row_num = 1
        """, '_q_'), use_bqstorage_api=True, progress_bar_type='tqdm_notebook')
    df_data.to_pickle('df_data.pkl')
else:
    df_data = pd.read_pickle('df_data.pkl')

In [ ]:
folds = range(1)

if False:
    df_test = pd.read_gbq((f"""
        SELECT row_id, user_id, content_id, answered_correctly,
        FROM riiid-caleb.data.train t
        WHERE content_type_id = 0
        AND fold in ({(', '.join(list(map(str, folds))))}) 
        ORDER BY user_id, content_id
        """, '_q_'), use_bqstorage_api=True, progress_bar_type='tqdm_notebook')
    df_data.to_pickle('df_test.pkl')
else:
    df_test = pd.read_pickle('df_test.pkl')

In [ ]:
# keeping out unseen users and questions to start with
df_test = df_test[df_test.user_id.isin(df_data.user_id)
                    & df_test.content_id.isin(df_data.content_id)]

In [ ]:
def categorify(df_train, df_valid, cat_cols):
    for col in cat_cols:
        df_train[col] = df_train[col].astype('category')
        df_valid[col] = pd.Categorical(df_valid[col], df_train[col].cat.categories)

    return df_train.sample(frac=1, random_state=42), df_valid    

In [ ]:
cat_cols = ['user_id', 'content_id']

In [ ]:
df_train, df_valid = categorify(df_data, df_test, cat_cols)

* For training / testing, create collaborative filtering features based on task_container_ids less than those included in folds in order to mimic the train / actual test scenario, where all of the task container_ids in the test set are after tnose in the train set.
* Then update the collaborative filtering features to be based on all of the records, but use the model trained based on cf only prior to 
* Could have a feature that represents that delta in task_container_ids or rows from the collaborative filtering feature.
* Would it be better to 


## Tensorflow Collaborative Filtering

In [ ]:
def get_ds(df, batch_size=1024, repeat=True):
    ds = tf.data.Dataset.from_tensor_slices(
        (pd.concat([df[col].cat.codes for col in cat_cols], axis=1),
         df.answered_correctly)
        )
    ds = ds.shuffle(int(1e6))
    ds = ds.repeat() if repeat else ds
    ds = ds.batch(batch_size)
    return ds.prefetch(AUTO)

In [ ]:
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    
    if type(value) != type(list()):
        value = [value]

    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [ ]:
def serialize_example(rec):
    
    feature = {
        'row_id': _int64_feature(rec['row_id']),
        'features': _int64_feature([rec['user_id'], rec['content_id']]),
        'target': _int64_feature(rec['answered_correctly'])
    }

    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()

In [ ]:
def parse_example(example, test=False):
    features = {'row_id': tf.io.FixedLenFeature([], tf.int64),
                'features': tf.io.FixedLenFeature([2], tf.int64),
                'target': tf.io.FixedLenFeature([], tf.int64),
                }

    example = tf.io.parse_single_example(example, features)

    return example['features'], example['target']

In [ ]:
def preprocess(batch):
    batch_size = tf.shape(batch['target'])[0]
    n_features = tf.shape(batch['features'])[1]
    features = tf.reshape(batch['features'], (batch_size, n_features))
    labels = tf.reshape(batch['target'], (batch_size,))
    return features, labels

In [ ]:
def get_ds_tfrec(split, batch_size=4096, repeat=True):
    file_pat = f'gs://{BUCKET}/tfrecords-{split}*.tfrec'
    options = tf.data.Options()
    
    ds = (tf.data.Dataset.list_files(file_pat, shuffle=True)
          .with_options(options)
          .interleave(tf.data.TFRecordDataset, num_parallel_calls=AUTO)
          .map(parse_example, num_parallel_calls=AUTO)
          )
    
    ds = ds.shuffle(int(1e3))
    ds = ds.repeat() if repeat else ds
    ds = ds.batch(batch_size)
    # ds = ds.map(preprocess, num_parallel_calls=AUTO)
    return ds.prefetch(AUTO)

In [ ]:
parse_example(serialize_example(df_data.head().to_records()[0]))

In [ ]:
if False:

    split_dict = {
        'train': df_train,
        'valid': df_valid
    }

    for s in split_dict:
        df = split_dict[s]
        n_files = len(df) // 500000
        for col in cat_cols:
            df[col] = df[col].cat.codes

        for i, split in enumerate(np.array_split(df, n_files)):
            out_path = f'gs://{bucket.name}'
            filename = f'tfrecords-{s}-{i:02d}-{len(split)}.tfrec'
            record_file = f'{out_path}/{filename}'

            with tf.io.TFRecordWriter(record_file) as writer:
                for rec in tqdm(split.to_records()):
                    writer.write(serialize_example(rec))

In [ ]:
n_users = len(df_train.user_id.cat.categories)
n_questions = len(df_train.content_id.cat.categories)
EMBEDDING_SIZE = 50

class KnowledgeNet(tf.keras.Model):
    def __init__(self, n_users, n_questions, embedding_size, l2_reg=1e-6, **kwargs):
        super(KnowledgeNet, self).__init__(**kwargs)
        self.num_users = n_users
        self.num_questions = n_questions
        self.embedding_size = embedding_size
        self.l2_reg = l2_reg
        self.user_embedding = tf.keras.layers.Embedding(
            n_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=tf.keras.regularizers.l2(self.l2_reg),
            name='user_embedding'
        )
        self.user_bias = tf.keras.layers.Embedding(
            n_users, 1,
            name='user_bias'
        )
        self.question_embedding = tf.keras.layers.Embedding(
            n_questions,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=tf.keras.regularizers.l2(self.l2_reg),
            name='question_embedding'
        )
        self.question_bias = tf.keras.layers.Embedding(
            n_questions, 1,
            name='question_bias'
        )

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        question_vector = self.question_embedding(inputs[:, 1])
        question_bias = self.question_bias(inputs[:, 1])
#         dot_user_question = tf.tensordot(user_vector, question_vector, 0)
        dot_user_question = tf.reduce_sum(user_vector * question_vector, axis=1, keepdims=True)
        x = user_bias + question_bias + dot_user_question
        return tf.nn.sigmoid(x)

In [ ]:
with strategy.scope():
    model = KnowledgeNet(n_users, n_questions, EMBEDDING_SIZE)
    opt = tf.keras.optimizers.Adam(lr=.0001)
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    metrics = ['binary_accuracy', 'AUC']
    model.compile(loss=loss_fn, optimizer=opt, metrics=metrics)

In [ ]:
for b in get_ds_tfrec('train').take(1):
    inputs = b[0]

In [ ]:
tf.reduce_sum(model.user_embedding(tf.repeat(tf.expand_dims(inputs[:,0], -1), 3, axis=1)), axis=1)

In [ ]:
model.user_embedding

In [ ]:
history = model.fit(
    get_ds_tfrec('train'),
    steps_per_epoch=5215,
    epochs=5,
    validation_data=get_ds_tfrec('valid'),
    validation_steps=1024,
    verbose=1
)

In [ ]:
model.save('collab_model_all', include_optimizer=True)

In [ ]:
for f in Path('collab_model_all').glob('**/*'):
    if f.is_file():
        bucket.blob(str(f)).upload_from_filename(str(f))

In [ ]:
model = tf.saved_model.load('collab_model_all')

In [ ]:
weights = {c: df_data[c].cat.categories for c in cat_cols}
for v in model.variables:
    name = v.name.split('/')[1]
    weights[name] = v.numpy()
    weights[name] = np.append(weights[name], np.mean(weights[name], axis=0))

In [ ]:
def get_pred(u_code, c_code, logits=False):
    user_vector = weights['user_embedding'][u_code]
    user_bias = weights['user_bias'][u_code]
    question_vector = weights['question_embedding'][c_code]
    question_bias = weights['question_bias'][c_code]
    
    logit = np.squeeze((user_vector * question_vector).sum() + user_bias + question_bias)
    
    if logits:
        return logit
    else:
        return 1 / (1 + np.math.exp(-logit))

In [ ]:
np.save('weights_all.npy', weights, allow_pickle=True)

In [ ]:
weights = np.load('weights_all.npy', allow_pickle=True).item()

In [ ]:
bucket.blob('weights_all.npy').upload_from_filename('weights_all.npy')

In [ ]:
def get_code_cols(df):
    return pd.concat([df[col].cat.codes for col in cat_cols], axis=1)

In [ ]:
df_cat_cols = get_code_cols(df_train)

In [ ]:
tqdm.pandas()

In [ ]:
get_pred(43124, 9276, True)

In [ ]:
df_preds = df_cat_cols.progress_apply(lambda r: get_pred(*r, logits=True), axis=1)

In [ ]:
roc_auc_score(df_valid.answered_correctly, df_preds)

In [ ]:
folds = range(5)

df_test_all = pd.read_gbq((f"""
    SELECT row_id, user_id, content_id, answered_correctly,
    FROM data.train t
    WHERE content_type_id = 0
    AND fold in ({(', '.join(list(map(str, folds))))}) 
    ORDER BY user_id, content_id
    """, '_q_'), use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

In [ ]:
df_test_all.to_pickle('df_test_all.pkl')

In [ ]:
for c in cat_cols:
    df_test_all[c] = pd.Categorical(df_test_all[c], categories=weights[c])

In [ ]:
df_code_cols = get_code_cols(df_test_all)

In [ ]:
preds_all = df_code_cols.progress_apply(lambda r: get_pred(*r, logits=True), axis=1)

In [ ]:
df_test_all['pred_collab_logit'] = preds_all

In [ ]:
df_test_all[['row_id', 'pred_collab', 'pred_collab_logit']].to_pickle('df_pred_collab.pkl')

In [ ]:
bucket.blob('df_pred_collab.pkl').upload_from_filename('df_pred_collab.pkl')

In [ ]:
roc_auc_score(df_test_all.answered_correctly, preds_all)

## Custom Training Loop

In [ ]:
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE_GLOBAL = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

ds_trn_fit_iter = iter(strategy.experimental_distribute_dataset(get_ds_tfrec('train', batch_size=BATCH_SIZE_GLOBAL)))
ds_val_fit_iter = iter(strategy.experimental_distribute_dataset(get_ds_tfrec('valid', batch_size=BATCH_SIZE_GLOBAL)))

In [ ]:
with strategy.scope():
    model = KnowledgeNet(n_users, n_questions, EMBEDDING_SIZE)
    
    loss_fn = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    def compute_loss(targets, outputs):
        per_example_loss = loss_fn(targets, outputs)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=BATCH_SIZE_GLOBAL)
    
    optimizer = tf.keras.optimizers.Adam(lr=0.0001)
    # metrics_trn = [tf.keras.metrics.Mean(name='loss'), tf.keras.metrics.BinaryAccuracy('accuracy'), tf.keras.metrics.AUC(name='roc_auc')]
    # metrics_val = [tf.keras.metrics.Mean(name='val_loss'), tf.keras.metrics.BinaryAccuracy('val_accuracy'), tf.keras.metrics.AUC(name='val_roc_auc')]
    metrics_trn = [tf.keras.metrics.Mean(name='loss')]
    metrics_val = [tf.keras.metrics.Mean(name='val_loss')]

In [ ]:
@tf.function
def train_step(model, optimizer, loss_fn, metrics, ds_iter, steps_per_epoch):
    def train_step_fn(inputs, targets):
        with tf.GradientTape() as tape:
            outputs = model(inputs, training=True)
            loss = loss_fn(targets, outputs)
                            
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        for i, metric in enumerate(metrics):
            if i == 0:
                metric.update_state(loss)
            else:
                metric.update_state(targets, outputs)
    
    for _ in tf.range(steps_per_epoch):
        strategy.run(train_step_fn, next(ds_iter))
        
@tf.function
def val_step(model, loss_fn, metrics, ds_iter, steps_per_epoch):
    def val_step_fn(inputs, targets):
        outputs = model(inputs, training=False)
        loss = loss_fn(targets, outputs)
        
        for i, metric in enumerate(metrics):
            if i == 0:
                metric.update_state(loss)
            else:
                metric.update_state(targets, outputs)
    
    for _ in tf.range(steps_per_epoch):
        strategy.run(val_step_fn, next(ds_iter))

In [ ]:
epochs = 5
steps_per_epoch = 10000
val_steps = 100
results = {metric.name: [] for metric in metrics_trn + metrics_val}
results['duration'] = []

start = time.perf_counter()
for epoch in range(epochs):
    epoch_start = time.perf_counter()
    train_step(model, optimizer, compute_loss, metrics_trn, ds_trn_fit_iter, steps_per_epoch)    
    val_step(model, compute_loss, metrics_val, ds_val_fit_iter, val_steps)
    
    for metric in metrics_trn + metrics_val:
        results[metric.name].append(metric.result().numpy())
        metric.reset_states()
        
    results['duration'].append((time.perf_counter() - epoch_start))
    
    print(f'epoch {epoch:02d} - ', (', ').join([f'{k}: {v[-1]:0.4f}' for k,v in results.items()]))

end = time.perf_counter()
seconds = end - start
minutes = seconds / 60 + (seconds % 60) / 60
print(f'Total duration: {minutes:0.1f} minutes - {seconds / epochs:0.1f} seconds per epoch')